In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('final_data.csv', index_col=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 0 to 1499
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   repo_name       1500 non-null   object
 1   star            1500 non-null   object
 2   fork            1500 non-null   object
 3   watch           1500 non-null   object
 4   issue           1500 non-null   object
 5   tags            1500 non-null   object
 6   description     1480 non-null   object
 7   contributers    1500 non-null   object
 8   license         1500 non-null   object
 9   repo_url        1500 non-null   object
 10  most_used_lang  1297 non-null   object
dtypes: object(11)
memory usage: 140.6+ KB


In [3]:
data.head()

,repo_name,star,fork,watch,issue,tags,description,contributers,license,repo_url,most_used_lang
0,keras,47.9k,18.1k,2.1k,"2,940","['deep-learning', 'tensorflow', 'neural-networ...",Deep Learning for humans,49,View license,https://github.com/keras-team/keras,Python
1,scikit-learn,40.3k,19.6k,2.2k,"1,505","['machine-learning', 'python', 'statistics', '...",scikit-learn: machine learning in Python,108,View license,https://github.com/scikit-learn/scikit-learn,Python
2,PythonDataScienceHandbook,23.1k,9.9k,1.5k,65,"['scikit-learn', 'numpy', 'python', 'jupyter-n...",Python Data Science Handbook: full text in Jup...,0,View license,https://github.com/jakevdp/PythonDataScienceHa...,Jupyter Notebook
3,Probabilistic-Programming-and-Bayesian-Methods...,21k,6.6k,1.4k,127,"['bayesian-methods', 'pymc', 'mathematical-ana...","aka ""Bayesian Methods for Hackers"": An introdu...",0,MIT,https://github.com/CamDavidsonPilon/Probabilis...,Jupyter Notebook
4,Data-Science--Cheat-Sheet,18.4k,8.2k,1.5k,7,[],Cheat Sheets,0,Fetching contributors,https://github.com/abhat222/Data-Science--Chea...,NaN


## Cleaning

So, we can see in the info section that some of the rows in the `description` and `most_used_lang` are null. I think most used language feature affects the popularity of the repo because the projects using trending technologies is the project most viewed by people. Let's see the popularity of repositories which has no language.

In [6]:
data[data['most_used_lang'].isnull()][['star', 'fork', 'most_used_lang']].head(10)

,star,fork,most_used_lang
4,18.4k,8.2k,NaN
8,14.7k,5.3k,NaN
11,12.1k,3.4k,NaN
16,10.3k,2.5k,NaN
17,9.9k,1.8k,NaN
18,9.7k,2.1k,NaN
20,8.7k,2.1k,NaN
22,7.8k,1.1k,NaN
26,7.4k,1.5k,NaN
30,6.9k,1.7k,NaN


Above, we can see that without having any language these repositories has many `stars` and `forks`. Then these repositories must share some resources or any useful information like this [here](https://github.com/loveunk/Deep-learning-books). So, I think replacing is the best option then dropping.

In [7]:
data['most_used_lang'] = data['most_used_lang'].fillna('no language', inplace=False)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 0 to 1499
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   repo_name       1500 non-null   object
 1   star            1500 non-null   object
 2   fork            1500 non-null   object
 3   watch           1500 non-null   object
 4   issue           1500 non-null   object
 5   tags            1500 non-null   object
 6   description     1480 non-null   object
 7   contributers    1500 non-null   object
 8   license         1500 non-null   object
 9   repo_url        1500 non-null   object
 10  most_used_lang  1500 non-null   object
dtypes: object(11)
memory usage: 140.6+ KB


We can see that there is rows which has no any `description`. We know having description of repo makes it more attractive or easy for other's to understand the project. It will affects the popularity somehow.

In [8]:
print(data[data['description'].isnull()][['most_used_lang', 'repo_url', 'star']])

     most_used_lang                                           repo_url   star
202             C++  https://github.com/oreillymedia/Learning-OpenC...   1.4k
214          Python  https://github.com/MicrocontrollersAndMore/Ope...    405
219            Java                   https://github.com/xikuqi/OpenCV    317
234             C++  https://github.com/MicrocontrollersAndMore/Ope...    217
242             C++         https://github.com/saki4510t/OpenCVwithUVC    154
245          Python  https://github.com/MicrocontrollersAndMore/Ras...    143
253          Python  https://github.com/MicrocontrollersAndMore/Ope...    128
269             C++  https://github.com/MicrocontrollersAndMore/Ope...    105
291             C++  https://github.com/MicrocontrollersAndMore/Ope...     74
356             CSS  https://github.com/handong1587/handong1587.git...     3k
428          Python   https://github.com/martinarjovsky/WassersteinGAN   2.6k
449          Python            https://github.com/hanzhanggit/St

We can see above that the non-descripted repo also has popularity. So, I think other than removing them I will replace it with `''`.

In [ ]:
data['description'] = data['description'].fillna('', inplace=False)
len(data[data['description'].isnull()])

In [ ]:
data.info()

## Visualization